HMMs Library 
============================
#### (Discrete & Continuous hidden markov models )

The document contain the tutorial ( usage explained by example ) for the hidden markov models library [link to pip].
* [The **first** part](#dthmm) will cover disrete-time hidden markov model (**DtHMM**)
* [The **second** part](#cthmm) will be dedicated to continuous-time hidden markov model (**CtHMM**)
* [The **third** part](#conv) will compare the convergences of **both** models

If you are not familiar with the hidden markov model theory, We recommend ...
%todo: refer to DP theory, github, sources


<a id='dthmm'></a>
Part 1: Discrete Time Hidden Markov Model
---------------------------------------------------

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import hmms

%matplotlib inline

### Construct DtHMM 
You can directly initialize the DtHMM by passing the **model parameters**.  

We will create simple DtHMM of two hidden states and three output variables.

In [ ]:
# A is the matrix of transition probabilities from state [row] to state [column].
A = np.array([[0.9,0.1],[0.4,0.6]])
# B is the matrix of probabilities that the state [row] will emmit output variable [column].
B = np.array([[0.9,0.08,0.02],[0.2,0.5,0.3]])
# Pi is the vector of initial state probabilities.   
Pi = np.array( [0.8,0.2] )

# Create DtHMM by given parameters.
dhmm = hmms.DtHMM(A,B,Pi)

Or you can initialize it by **random parameters**. Passing the number of hidden states and output variables. 

In [ ]:
dhmm_random = hmms.DtHMM.random(2,3)

### Save & Read from File
Once you have created the model you can **save** its parameters in file simply by calling *save_params* method.  

In [ ]:
dhmm.save_params("Models/hello_dthmm") 

The method stored the parameters in *.npz* format.  

The saved file can be later used to **read** parametrs for model initialization.

In [ ]:
dhmm_from_file = hmms.DtHMM.from_file( "Models/hello_dthmm.npz" )

### Set & Get Parameters
Later you can always **set** parameters with triple of methods corresponding to the constructors.

In [ ]:
dhmm.set_params(A,B,Pi)
dhmm.set_params_random(2,3)
dhmm.set_params_from_file( "Models/hello_dthmm.npz" )

You can **get** parameters by calling them separately,

In [ ]:
dhmm.a, dhmm.b, dhmm.pi

or **get** them **all** together as the triple.

In [ ]:
(A,B,Pi) = dhmm.params

### Generate Random State and Emission Sequence
 Now we can use our model to generate state and emission sequence.   
 The model will randomly choose which transition or emission will happen, taking into consideration the parameters we have previously defined.


In [ ]:
seq_len = 20
s_seq, e_seq = dhmm.generate( seq_len )

#resize plot
plt.rcParams['figure.figsize'] = [20,20]

hmms.plot_hmm( s_seq, e_seq )

### Find Most Likely State Sequence
If we have the model parameters and emission sequence, we can find the most probable state sequence that would generate it. Notice, that it can be different, than the actual sequence that has generated the emissions.  
We will use Viterbi algorithm for the calculation.


In [ ]:
( prob, s_seq ) =  dhmm.viterbi( e_seq )
# Let's print the most likely state sequence, it can be same or differ from the sequence above.
hmms.plot_hmm( s_seq, e_seq )

The *prob* parameter store the probability of the sequence.  
All the probabilities in the library are stored in the logarithm of their actual value. As the number of possible sequences grows exponentialy by it length, it could easily lead to float underflow.  
You can easily transform it to the normal scale value applying *exp* function. 

In [ ]:
np.exp( prob )

### The Probability of the Emission Sequence
We can compute the probabilityoft the model generating the emission sequence.

In [ ]:
np.exp( dhmm.emission_estimate( e_seq ) )

### Generate Artificial Dataset

You can easily generate many sequences in once by using the generate_data function.
The generated emission sequences are in the form that is suitable for training of parameters. You can switch times=True, if you want to generate also the corresponding equidistant time sequences.

In [ ]:
seq_num= 3   #number of data sequences
seq_len= 10  #length of each sequence

dhmm.generate_data( (seq_num,seq_len) )

### Create Hidden Markov Model with Random Parameters
We will use *get_random_parameters* function to generate parameters. It takes arguments for number of hidden states and number of observation variables.

In [ ]:
dhmm_r = hmms.DtHMM.random( 2,3 )
# We can print all the parameters.
hmms.print_parameters( dhmm_r )

### Parameters Estimation - Baum Welch Algorithm
We usually do not know the real parameters of the model.  
Here we will have several output variables (emissions) sequences and we will show, how to estimate the model parameters

Let's start by creating some artifficial data. We will use the previously defined model for it.


In [ ]:
seq_num = 1
seq_len = 200
data = np.empty( (seq_num, seq_len), dtype=int )

for i in range(seq_num):
    _, data[i] = dhmm.generate( seq_len )

Let's compare the dataset likelihood estimation of model used for generating the data and the random parameters model. 

In [ ]:
print( "Generator model:" , np.exp( dhmm.data_estimate(data)   ) )
print( "Random model:   " ,np.exp( dhmm_r.data_estimate(data) ) )

Most likely the probability that the data was generated by random model is extremly low.

Now we can take the random model and reestimate it to fit the data better.

In [ ]:
dhmm_r.baum_welch( data, 10 )
print( "Reestimated model after 10 iterations:   " ,np.exp( dhmm_r.data_estimate(data) ) )

The probability of the reestimated model should now be similiar (possibly even better) that the generator's model. If it is not, you can try to run the estimation procedure more time at different randomly generated models. It could happen that the estimation fall in the local optima.

If you are satisfied with the results, you can run some more iteration to fine-tune it.

In [ ]:
dhmm_r.baum_welch( data, 1 )
print( "Reestimated model after 110 iterations:   " ,np.exp( dhmm_r.data_estimate(data) ) )

In [ ]:
dhmm.baum_welch( data, 10 )  
print( "Gnerator model after 10 iterations:   " ,np.exp( dhmm_r.data_estimate(data) ) )

In [ ]:
hmms.print_parameters( dhmm_r )
hmms.print_parameters( dhmm )

In [ ]:
hmm_r = hmms.DtHMM.random(2,3) 
real = hmm.data_estimate(data)
out = hmm_r.baum_welch_graph( data, 1 )
print( np.exp(out) )
plt.plot( out[2:] / real  )
plt.show()

In [ ]:
hmm.save_params("Models/my_file")
hmf = hmms.DtHMM.from_file("Models/my_file.npz")

In [ ]:
hmms.print_parameters( hmf )
hmms.print_parameters( hmm )

<a id='cthmm'></a>
Part 2: Continuous Time Hidden Markov Model
-----------------------------------------------------

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import hmms

%matplotlib inline

### Construct CtHMM 
Construction of CtHMM is similar to the discrete model. 

You can directly initialize the CtHMM by passing the **model parameters**.  
We will create simple CtHMM of three hidden states and three output variables.

In [ ]:
# Q is the matrix of transition rates from state [row] to state [column].
Q = np.array( [[-0.375,0.125,0.25],[0.25,-0.5,0.25],[0.25,0.125,-0.375]] )
# B is the matrix of probabilities that the state [row] will emmit output variable [column].
B = np.array( [[0.8,0.05,0.15],[0.05,0.9,0.05],[0.2,0.05,0.75]] )
# Pi is the vector of initial state probabilities.   
Pi = np.array( [0.6,0,0.4] )

# Create CtHMM by given parameters.
chmm = hmms.CtHMM(Q,B,Pi)

Or you can initialize it by **random parameters**. Passing the number of hidden states and output variables. 

In [ ]:
chmm_random = hmms.CtHMM.random(3,3)

### Save & Read from File

Once you have created the model you can save its parameters in file simply by calling save_params method.

In [ ]:
chmm.save_params("Models/hello_cthmm") 

The method stored the parameters in .npz format.
The saved file can be later used to read parametrs for model initialization.

In [ ]:
chmm_from_file = hmms.CtHMM.from_file( "Models/hello_cthmm.npz" )

### Set & Get Parameters
Later you can always set parameters with triple of methods corresponding to the constructors.

In [ ]:
chmm.set_params(Q,B,Pi)
chmm.set_params_random(3,3)
chmm.set_params_from_file( "Models/hello_cthmm.npz" )

You can **get** parameters by calling them separately,

In [ ]:
chmm.q, chmm.b, chmm.pi

or get them all together as the triple.

In [ ]:
(A,B,Pi) = chmm.params

### Generate Random Sequence
Now we can use our model to **generate** time, state and emission sequence.
The model will **randomly** choose which transition or emission will happen, taking into consideration the parameters we have previously defined.

The times are generated with **exponencial** waiting times, you can define the parameter of exponencial distribution as second optional parameter. 


In [ ]:
seq_len = 10
t_seq, s_seq, e_seq = chmm.generate( seq_len )

#resize plot
plt.rcParams['figure.figsize'] = [20,20]

hmms.plot_hmm( s_seq, e_seq, time=t_seq )

### Generate Artificial  Dataset
You can easily generate many sequences in once by using the *generate_data* function.  
The generated data are in the form that is suitable for training of parameters. 
You can switch *states=True*, if you want to generate also the corresponding state sequences.

TODO: The times are generated with **exponencial** waiting times, you can define the parameter of exponencial distribution as second optional parameter. 

In [ ]:
seq_num= 3   #number of data sequences
seq_len= 10  #length of each sequence

chmm.generate_data( (seq_num,seq_len) )

In [ ]:
import numpy
import scipy.linalg

def get_dthmm_params( cthmm ):
    """given the cthmm get parameters for dthmm"""
    A = scipy.linalg.expm( cthmm.q )  #the transition rate is set as the one time unit probabilities of continuos model
    B = cthmm.b
    Pi = cthmm.pi
    return (A,B,Pi)


In [ ]:
Q = numpy.array( [[-0.375,0.125,0.25],[0.25,-0.5,0.25],[0.25,0.125,-0.375]] )
B = numpy.array( [[0.8,0.05,0.15],[0.05,0.9,0.05],[0.2,0.05,0.75]] )
Pi = numpy.array( [0.6,0,0.4] )

In [ ]:
cthmm = hmms.CtHMM( Q,B,Pi )
dthmm = hmms.DtHMM( *get_dthmm_params(cthmm) )

In [ ]:
t, e = create_data( dthmm, (50,50) )

ct = hmms.CtHMM.random(3,3)
dt = hmms.DtHMM( *get_dthmm_params(ct) )
    
#hmms.print_parameters( dt )
#hmms.print_parameters( ct )

creal = cthmm.data_estimate(t,e)
dreal = dthmm.data_estimate(e)
print(creal)
print(dreal)

In [ ]:

out = dt.baum_welch_graph( e, 1 )
plt.plot( out[2:] / dreal  )

outc = ct.baum_welch_graph( t,e, 1 )
plt.plot( outc[2:] / dreal  )

plt.show()


In [ ]:

plt.plot( out[1:] / dreal  )
plt.plot( outc[1:] / dreal  )
#plt.savefig('100itchmmvsdtmm3.svg')
plt.show()


In [ ]:
cest = ct.data_estimate(t,e)
dest = dt.data_estimate(e)
print(cest)
print(dest)

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import hmms

%matplotlib inline
import numpy

Q = numpy.array( [[-0.375,0.125,0.25],[0.25,-0.5,0.25],[0.25,0.125,-0.375]] )
B = numpy.array( [[0.8,0.05,0.15],[0.05,0.9,0.05],[0.2,0.05,0.75]] )
Pi = numpy.array( [0.6,0,0.4] )
cthmm = hmms.CtHMM(Q,B,Pi)
#hmms.print_parameters(cthmm)

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import hmms

%matplotlib inline
import numpy

Q = numpy.array( [[-0.375,0.125,0.25],[0.25,-0.5,0.25],[0.25,0.125,-0.375]] )
B = numpy.array( [[0.8,0.05,0.15],[0.05,0.9,0.05],[0.2,0.05,0.75]] )
Pi = numpy.array( [0.6,0,0.4] )
cthmm = hmms.CtHMM(Q,B,Pi)
#hmms.print_parameters(cthmm)



datae = ( numpy.array([ [ 0,  1,  2,  3,  6,  8, 13, 14, 15, 16],
                           [ 0,  1,  2,  3,  5,  8, 10, 14, 17, 20],
                           [ 0,  6,  9, 14, 17, 21, 22, 25, 29, 30],
                           [ 0,  1,  2,  5,  8, 10, 11, 14, 16, 17],
                           [ 0,  1,  3,  5,  7,  9, 12, 15, 16, 22 ] ] ),
             numpy.array([ [0, 0, 0, 0, 2, 1, 1, 1, 2, 0],
                           [0, 0, 0, 1, 0, 0, 0, 0, 0, 2],
                           [0, 0, 1, 0, 2, 0, 0, 2, 2, 2],
                           [0, 0, 0, 2, 1, 2, 2, 2, 2, 0],
                           [2, 0, 0, 2, 0, 2, 2, 0, 0, 0] ] )
           )

data = ( numpy.array([ [ 0,  1,  2,  3,  4,  5, 6, 7, 8, 10],
                           [ 0,  1,  2,  3,  4,  5, 6, 7, 8, 9],
                           [ 0,  1,  2,  3,  4,  5, 6, 7, 8, 9],
                           [ 0,  1,  2,  3,  4,  5, 6, 7, 8, 9],
                           [ 0,  1,  2,  3,  4,  5, 6, 7, 8, 9] ] ),
             numpy.array([ [0, 0, 0, 0, 2, 1, 1, 1, 2, 0],
                           [0, 0, 0, 1, 0, 0, 0, 0, 0, 2],
                           [0, 0, 1, 0, 2, 0, 0, 2, 2, 2],
                           [0, 0, 0, 2, 1, 2, 2, 2, 2, 0],
                           [2, 0, 0, 2, 0, 2, 2, 0, 0, 0] ] )
           )


#data = ( numpy.array([[8,10]]) , numpy.array([[2,0]])  )

datae2 = ( numpy.array([ [ 0,  1,  2,  3,  4,  5, 6, 7, 8, 9],
                           [ 0,  1,  2,  3,  4,  5, 6, 7, 8, 9],
                           [ 0,  1,  2,  3,  4,  5, 6, 7, 8, 9],
                           [ 0,  1,  2,  3,  4,  5, 6, 7, 8, 9],
                           [ 0,  1,  2,  3,  4,  5, 6, 7, 8, 9] ] ),
             numpy.array([ [0, 0, 0, 0, 2, 1, 1, 1, 2, 0],
                           [0, 0, 0, 1, 0, 0, 0, 0, 0, 2],
                           [0, 0, 1, 0, 2, 0, 0, 2, 2, 2],
                           [0, 0, 0, 2, 1, 2, 2, 2, 2, 0],
                           [2, 0, 0, 2, 0, 2, 2, 0, 0, 0] ] )
           )


In [ ]:
#for i in range(5):
#    hmms.plot_hmm( data[0][i], data[1][i], time = data[0][i] )

In [ ]:
t,e = data
te,ee = datae


cthmme = hmms.CtHMM( *cthmm.params )

ceste = cthmme.data_estimate(te,ee)

cest = cthmm.data_estimate(t,e)
( cest, ceste )

In [ ]:
#cthmm.baum_welch( t,e,20 )
outc = cthmm.baum_welch_graph( t,e,5 )
plt.plot( outc / cest  )
plt.show()

outce = cthmme.baum_welch_graph( te,ee,20 )
plt.plot( outce / ceste  )
plt.show()



In [ ]:
hmms.print_parameters(cthmme)

In [ ]:
cthmm.data_estimate(t,e)

In [ ]:
outc

In [ ]:
t


In [ ]:
e